# Mixture-of-Agents: Automated MLB Game Recap Generator
### Using Multiple AI Models with CrewAI and Groq

**Author:** Abdelbasset Djamai

---

This project builds a system where multiple AI agents work together to create better content than any single model could produce alone.

The concept is inspired by the [Mixture of Agents (MoA) paper](https://arxiv.org/pdf/2406.04692) - basically, instead of relying on one AI model, I'm using several different models (LLaMA, Gemma, Mixtral) to each write their own version of an MLB game recap. Then, a final "editor" agent reviews all three versions and combines the best parts into one polished article.

Why baseball? Well, game recaps need to be factual, engaging, and structured. It's a perfect test case for this multi-agent approach.

## Setup and Dependencies

In [ ]:
# Required libraries for the project
import os
import statsapi  # MLB stats API wrapper
import datetime
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np
from crewai_tools import tool
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq

## LLM Configuration

Setting up different language models via Groq's API. Each model has its own strengths:
- **LLaMA 70B**: The "smart" model - great for complex reasoning and editing
- **LLaMA 8B, Gemma 9B, Mixtral**: The "writers" - each with different writing styles

You'll need a Groq API key (get one [here](https://console.groq.com/keys)) and set it as an environment variable.

In [ ]:
# Initialize the different LLMs I'll be using
editor_llm = ChatGroq(model_name="llama3-70b-8192")  # The smart one for research & editing
writer_llm_1 = ChatGroq(model_name="llama3-8b-8192")  # Writer #1
writer_llm_2 = ChatGroq(model_name="gemma2-9b-it")    # Writer #2
writer_llm_3 = ChatGroq(model_name="mixtral-8x7b-32768")  # Writer #3

## Custom Tools for Data Collection

These tools allow my AI agents to fetch real MLB data. I'm using the MLB-StatsAPI to pull game information and player statistics. Think of these as the agents' research assistants!

In [ ]:
@tool
def get_game_info(game_date: str, team_name: str) -> str:
    """Gets high-level information on an MLB game.

    Params:
    game_date: The date of the game of interest, in the form "yyyy-mm-dd".
    team_name: MLB team name. Both full name (e.g. "New York Yankees") or nickname ("Yankees") are valid. If multiple teams are mentioned, use the first one
    """
    sched = statsapi.schedule(start_date=game_date,end_date=game_date)
    sched_df = pd.DataFrame(sched)
    game_info_df = sched_df[sched_df['summary'].str.contains(team_name, case=False, na=False)]

    game_id = str(game_info_df.game_id.tolist()[0])
    home_team = game_info_df.home_name.tolist()[0]
    home_score = game_info_df.home_score.tolist()[0]
    away_team = game_info_df.away_name.tolist()[0]
    away_score = game_info_df.away_score.tolist()[0]
    winning_team = game_info_df.winning_team.tolist()[0]
    series_status = game_info_df.series_status.tolist()[0]

    game_info = '''
        Game ID: {game_id}
        Home Team: {home_team}
        Home Score: {home_score}
        Away Team: {away_team}
        Away Score: {away_score}
        Winning Team: {winning_team}
        Series Status: {series_status}
    '''.format(game_id = game_id, home_team = home_team, home_score = home_score,
               away_team = away_team, away_score = away_score, \
                series_status = series_status, winning_team = winning_team)

    return game_info


@tool
def get_batting_stats(game_id: str) -> str:
    """Gets player boxscore batting stats for a particular MLB game

    Params:
    game_id: The 6-digit ID of the game
    """
    boxscores=statsapi.boxscore_data(game_id)
    player_info_df = pd.DataFrame(boxscores['playerInfo']).T.reset_index()

    away_batters_box = pd.DataFrame(boxscores['awayBatters']).iloc[1:]
    away_batters_box['team_name'] = boxscores['teamInfo']['away']['teamName']

    home_batters_box = pd.DataFrame(boxscores['homeBatters']).iloc[1:]
    home_batters_box['team_name'] = boxscores['teamInfo']['home']['teamName']

    batters_box_df = pd.concat([away_batters_box, home_batters_box]).merge(player_info_df, left_on = 'name', right_on = 'boxscoreName')
    return str(batters_box_df[['team_name','fullName','position','ab','r','h','hr','rbi','bb','sb']])


@tool
def get_pitching_stats(game_id: str) -> str:
    """Gets player boxscore pitching stats for a particular MLB game

    Params:
    game_id: The 6-digit ID of the game
    """
    boxscores=statsapi.boxscore_data(game_id)
    player_info_df = pd.DataFrame(boxscores['playerInfo']).T.reset_index()

    away_pitchers_box = pd.DataFrame(boxscores['awayPitchers']).iloc[1:]
    away_pitchers_box['team_name'] = boxscores['teamInfo']['away']['teamName']

    home_pitchers_box = pd.DataFrame(boxscores['homePitchers']).iloc[1:]
    home_pitchers_box['team_name'] = boxscores['teamInfo']['home']['teamName']

    pitchers_box_df = pd.concat([away_pitchers_box,home_pitchers_box]).merge(player_info_df, left_on = 'name', right_on = 'boxscoreName')
    return str(pitchers_box_df[['team_name','fullName','ip','h','r','er','bb','k','note']])


## Building the AI Team

Here's where it gets interesting. I'm creating a team of specialized agents:
1. **Researcher** - Finds the game we're talking about
2. **Statistician** - Pulls all the numbers and player stats
3. **Three Writers** - Each writes their own version of the recap using different models
4. **Editor** - Takes the best parts from all three drafts to create the final piece

### The Mixture of Agents Architecture

My implementation uses a simple 2-layer structure:
- **Layer 1**: Three writer agents independently create game recaps
- **Layer 2**: One editor agent synthesizes the best version

This design is intentionally simple, but you could easily expand it with more layers or specialized agents.

![Mixture of Agents Diagram](mixture_of_agents_diagram.png)

In [ ]:
# Agent 1: The Researcher - finds the game
researcher = Agent(
    llm=editor_llm,
    role="MLB Researcher",
    goal="Identify and return info for the MLB game related to the user prompt by returning the exact results of the get_game_info tool",
    backstory="An MLB researcher that identifies games for statisticians to analyze stats from",
    tools=[get_game_info],
    verbose=True,
    allow_delegation=False
)

# Agent 2: The Statistician - gathers all the data
statistician = Agent(
    llm=editor_llm,
    role="MLB Statistician",
    goal="Retrieve player batting and pitching stats for the game identified by the MLB Researcher",
    backstory="An MLB Statistician analyzing player boxscore stats for the relevant game",
    tools=[get_batting_stats, get_pitching_stats],
    verbose=True,
    allow_delegation=False
)

# Agents 3-5: The Writers - each using a different model
writer_agent_1 = Agent(
    llm=writer_llm_1,
    role="MLB Writer",
    goal="Write a detailed game recap article using the provided game information and stats",
    backstory="An experienced and honest writer who does not make things up",
    tools=[],
    verbose=True,
    allow_delegation=False
)

writer_agent_2 = Agent(
    llm=writer_llm_2,
    role="MLB Writer",
    goal="Write a detailed game recap article using the provided game information and stats",
    backstory="An experienced and honest writer who does not make things up",
    tools=[],
    verbose=True,
    allow_delegation=False
)

writer_agent_3 = Agent(
    llm=writer_llm_3,
    role="MLB Writer",
    goal="Write a detailed game recap article using the provided game information and stats",
    backstory="An experienced and honest writer who does not make things up",
    tools=[],
    verbose=True,
    allow_delegation=False
)

# Agent 6: The Editor - combines the best elements
editor = Agent(
    llm=editor_llm,
    role="MLB Editor",
    goal="Edit multiple game recap articles to create the best final product.",
    backstory="An experienced editor that excels at taking the best parts of multiple texts to create the best final product",
    tools=[],
    verbose=True,
    allow_delegation=False
)

## Defining the Workflow

Now I need to tell each agent exactly what to do. Tasks in CrewAI define the specific work each agent performs and how they depend on each other.

*Note: I had to disable async execution due to a CrewAI update (7/29/24) - keeping this sequential for now.*

In [ ]:
# Task 1: Find the game
research_task = Task(
    description='''
    Identify the correct game related to the user prompt and return game info using the get_game_info tool.
    Unless a specific date is provided in the user prompt, use {default_date} as the game date
    User prompt: {user_prompt}
    ''',
    expected_output='High-level information of the relevant MLB game',
    agent=researcher
)

# Task 2: Get batting statistics (runs after research_task)
batting_stats_task = Task(
    description='Retrieve ONLY boxscore batting stats for the relevant MLB game',
    expected_output='A table of batting boxscore stats',
    agent=statistician,
    dependencies=[research_task],
    context=[research_task]
)

# Task 3: Get pitching statistics (runs after research_task)
pitching_stats_task = Task(
    description='Retrieve ONLY boxscore pitching stats for the relevant MLB game',
    expected_output='A table of pitching boxscore stats',
    agent=statistician,
    dependencies=[research_task],
    context=[research_task]
)

# Tasks 4-6: Three writers create independent drafts
writing_task_1 = Task(
    description='''
    Write a game recap article using the provided game information and stats.
    Key instructions:
    - Include things like final score, top performers and winning/losing pitcher.
    - Use ONLY the provided data and DO NOT make up any information, such as specific innings when events occurred, that isn't explicitly from the provided input.
    - Do not print the box score
    ''',
    expected_output='An MLB game recap article',
    agent=writer_agent_1,
    dependencies=[research_task, batting_stats_task, pitching_stats_task],
    context=[research_task, batting_stats_task, pitching_stats_task]
)

writing_task_2 = Task(
    description='''
    Write a game recap article using the provided game information and stats.
    Key instructions:
    - Include things like final score, top performers and winning/losing pitcher.
    - Use ONLY the provided data and DO NOT make up any information, such as specific innings when events occurred, that isn't explicitly from the provided input.
    - Do not print the box score
    ''',
    expected_output='An MLB game recap article',
    agent=writer_agent_2,
    dependencies=[research_task, batting_stats_task, pitching_stats_task],
    context=[research_task, batting_stats_task, pitching_stats_task]
)

writing_task_3 = Task(
    description='''
    Write a succinct game recap article using the provided game information and stats.
    Key instructions:
    - Structure with the following sections:
          - Introduction (game result, winning/losing pitchers, top performer on the winning team)
          - Other key performers on the winning team
          - Key performers on the losing team
          - Conclusion (including series result)
    - Use ONLY the provided data and DO NOT make up any information, such as specific innings when events occurred, that isn't explicitly from the provided input.
    - Do not print the box score or write out the section names
    ''',
    expected_output='An MLB game recap article',
    agent=writer_agent_3,
    dependencies=[research_task, batting_stats_task, pitching_stats_task],
    context=[research_task, batting_stats_task, pitching_stats_task]
)

# Task 7: Editor synthesizes the final version
editing_task = Task(
    description='''
    You will be provided three game recap articles from multiple writers. Take the best of
    all three to output the optimal final article.

    Pay close attention to the original instructions:

    Key instructions:
        - Structure with the following sections:
          - Introduction (game result, winning/losing pitchers, top performer on the winning team)
          - Other key performers on the winning team
          - Key performers on the losing team
          - Conclusion (including series result)
        - Use ONLY the provided data and DO NOT make up any information, such as specific innings when events occurred, that isn't explicitly from the provided input.
        - Do not print the box score or write out the section names

    It is especially important that no false information, such as any inning or the inning in which an event occured,
    is present in the final product. If a piece of information is present in one article and not the others, it is probably false
    ''',
    expected_output='An MLB game recap article',
    agent=editor,
    dependencies=[research_task, batting_stats_task, pitching_stats_task],
    context=[research_task, batting_stats_task, pitching_stats_task]
)

## Running the System

Time to put it all together! The Crew orchestrates all the agents and makes sure everything runs in the right order.

In [ ]:
# Assemble the crew with all agents and tasks
my_crew = Crew(
    agents=[researcher, statistician, writer_agent_1, writer_agent_2, writer_agent_3, editor],
    tasks=[
        research_task,
        batting_stats_task, pitching_stats_task,
        writing_task_1, writing_task_2, writing_task_3,
        editing_task
        ],
    verbose=False
)

## Testing It Out

Let's generate a recap for a Yankees game from July 2024. I'm curious to see how the three different models approach writing the same story, and what the editor decides to keep from each version.

In [ ]:
# Set up the prompt
my_prompt = 'Write a recap of the Yankees game on July 14, 2024'
yesterday = datetime.now().date() - timedelta(1)  # Fallback to yesterday if no date specified

# Execute the workflow
final_output = my_crew.kickoff(inputs={"user_prompt": my_prompt, "default_date": str(yesterday)})

## Results and Observations

What's really cool about this approach is seeing how different models make different mistakes:
- Some hallucinate specific innings when events happened (data we don't have)
- Others misinterpret the stats (like saying the wrong pitcher got the win)
- A few have formatting issues or miss important details

But the editor agent? It's smart enough to catch these inconsistencies. When something appears in only one draft and not the others, it knows to be suspicious. The final output is more accurate and polished than any single version.

Here's the final recap:

In [ ]:
print(final_output)

### The End!